## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
###   VACATION SEARCH 

# Dependencies and Setup
import pandas as pd
import requests
#%pip install gmaps
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pozoblanco,ES,38.3791,-4.8483,82.98,20,0,6.02,clear sky
1,1,Manta,EC,-0.9500,-80.7333,76.50,75,98,10.83,overcast clouds
2,2,Bandarbeyla,SO,9.4942,50.8122,76.82,84,100,29.37,overcast clouds
3,3,Dawson Creek,CA,55.7666,-120.2362,51.10,95,100,4.47,overcast clouds
4,4,Ternate,ID,0.8000,127.4000,79.70,78,100,4.38,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pozoblanco,ES,38.3791,-4.8483,82.98,20,0,6.02,clear sky
1,1,Manta,EC,-0.9500,-80.7333,76.50,75,98,10.83,overcast clouds
2,2,Bandarbeyla,SO,9.4942,50.8122,76.82,84,100,29.37,overcast clouds
4,4,Ternate,ID,0.8000,127.4000,79.70,78,100,4.38,moderate rain
7,7,Jamestown,US,42.0970,-79.2353,78.12,65,1,4.00,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                257
City                   257
Country                256
Lat                    257
Lng                    257
Max Temp               257
Humidity               257
Cloudiness             257
Wind Speed             257
Current Description    257
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna(how = 'any')
preferred_cities_df.count()

City_ID                256
City                   256
Country                256
Lat                    256
Lng                    256
Max Temp               256
Humidity               256
Cloudiness             256
Wind Speed             256
Current Description    256
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Pozoblanco,ES,82.98,clear sky,38.3791,-4.8483,
1,Manta,EC,76.50,overcast clouds,-0.9500,-80.7333,
2,Bandarbeyla,SO,76.82,overcast clouds,9.4942,50.8122,
4,Ternate,ID,79.70,moderate rain,0.8000,127.4000,
7,Jamestown,US,78.12,clear sky,42.0970,-79.2353,
8,Lompoc,US,87.01,clear sky,34.6391,-120.4579,
10,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,
14,Bathsheba,BB,86.61,scattered clouds,13.2167,-59.5167,
18,Amapa,BR,83.59,clear sky,1.0000,-52.0000,
19,Itarema,BR,81.77,broken clouds,-2.9248,-39.9167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    #print(hotels)
    #break
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        #print("ok")
    except Exception as e:
        #print(e)
        #print(hotels)
        
                
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
#Checking the output
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Pozoblanco,ES,82.98,clear sky,38.3791,-4.8483,Hotel Nomada
1,Manta,EC,76.50,overcast clouds,-0.9500,-80.7333,Balandra Hotel
2,Bandarbeyla,SO,76.82,overcast clouds,9.4942,50.8122,JABIR HOTEL
4,Ternate,ID,79.70,moderate rain,0.8000,127.4000,Grand Dafam Bela Ternate
7,Jamestown,US,78.12,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown


In [10]:
hotel_df.count()

City                   256
Country                256
Max Temp               256
Current Description    256
Lat                    256
Lng                    256
Hotel Name             256
dtype: int64

In [11]:
#problem starts here 
#Fill empty rows with Nans and remove Nans.
import numpy as np
hotel_df_nan = hotel_df[hotel_df['Hotel Name']==''] = np.nan

In [12]:
# 7b. If there were an empty rows, would have been dropped with: Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how = 'any')
clean_hotel_df.count()

City                   242
Country                242
Max Temp               242
Current Description    242
Lat                    242
Lng                    242
Hotel Name             242
dtype: int64

In [13]:
#7c. Checking final dataframe before exporting.
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Pozoblanco,ES,82.98,clear sky,38.3791,-4.8483,Hotel Nomada
1,Manta,EC,76.50,overcast clouds,-0.9500,-80.7333,Balandra Hotel
2,Bandarbeyla,SO,76.82,overcast clouds,9.4942,50.8122,JABIR HOTEL
4,Ternate,ID,79.70,moderate rain,0.8000,127.4000,Grand Dafam Bela Ternate
7,Jamestown,US,78.12,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
8,Lompoc,US,87.01,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
10,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort
14,Bathsheba,BB,86.61,scattered clouds,13.2167,-59.5167,Atlantis Hotel
19,Itarema,BR,81.77,broken clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
39,Vicosa,BR,79.18,overcast clouds,-20.7539,-42.8819,Vivant Residence


In [14]:
# 8a. Create the output File (CSV)
 
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

clean_hotel_df.count()


City                   242
Country                242
Max Temp               242
Current Description    242
Lat                    242
Lng                    242
Hotel Name             242
dtype: int64

In [16]:
#NEED some help starting here  - 

# 11a. Add a marker layer for each city to the map. 

max_temp = clean_hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
            max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig





Figure(layout=FigureLayout(height='420px'))

In [17]:



#use as example  
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))